In [1]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import os
import json
from tqdm import tqdm

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [2]:
from bcnf.models.cnf import CondRealNVP_v2
from bcnf.utils import get_dir
from bcnf.utils import load_config, load_data, sub_root_path, inn_nll_loss
from bcnf.eval.calibration import compute_y_hat_ranks, compute_CDF_residuals

from bcnf.train.trainer_data_handler import TrainerDataHandler
from bcnf.train.utils import get_data_type

## Model

In [3]:
MODEL_NAME = f'trajectory_TRF_small'

In [4]:
with open(os.path.join(get_dir('models', 'bcnf-models', MODEL_NAME), 'config.json'), 'r') as f:
    config = load_config(json.load(f)['config_path'])
    config['data']['path'] = sub_root_path(config['data']['path']).replace('/train', '/test')
    print(config['data']['path'])

cnf = CondRealNVP_v2.from_config(config).to(device)

cnf.load_state_dict(torch.load(os.path.join(get_dir('models', 'bcnf-models', MODEL_NAME), "state_dict.pt")))

cnf.eval();

/home/psaegert/Projects/bcnf/data/bcnf-data/fixed_data_render_2s_15FPS/test


In [5]:
print(f'Number of parameters: {cnf.n_params:,}')

Number of parameters: 128,129


In [6]:
parameter_selection = cnf.parameter_index_mapping.parameters

## Data

In [7]:
T = 2
dt = 1/15

In [8]:
data = load_data(sub_root_path(config['data']['path']).replace('/train', '/test'), verbose=True, keep_output_type='trajectories')
# data = load_data(sub_root_path(config['data']['path']).replace('/train', '/test'), verbose=True, keep_output_type='videos')

Loading data from directory: 100%|██████████| 1/1 [00:01<00:00,  1.32s/it, file=fixed_data_render_2s_15FPS_5.pkl]


In [9]:
y, conditions = TrainerDataHandler().get_data_for_training(
    config,
    cnf.parameter_index_mapping,
    dtype=get_data_type(dtype=config["global"]["dtype"]),
    return_tensor_dataset=False)

print(f'{y.shape=}')
for condition in conditions:
    print(f'{condition.shape=}')

y.shape=torch.Size([1000, 19])
condition.shape=torch.Size([1000, 30, 3])


## Code Space

In [10]:
z = torch.empty(y.shape)
batch_size = 32

with torch.no_grad():
    # for i in tqdm(range(0, len(y), batch_size)):
    #     z[i:i+batch_size] = cnf.forward(y[i:i+batch_size].to(device), *[c[i:i+batch_size].to(device) for c in conditions], log_det_J=True)
    z = cnf.forward(y.to(device), *conditions, log_det_J=True)

In [11]:
z.shape

torch.Size([1000, 19])

In [12]:
inn_nll_loss(z, log_det_J=cnf.log_det_J)

tensor(-46.1068, device='cuda:0')